***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [56]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now() - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting_filter = (transporting['DenTinh'] == 'Hà Nội') | (transporting['DenTinh'] == 'Hồ Chí Minh')
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting.loc[transporting_filter, 'N+'] = transporting.loc[transporting_filter]['N0']+ pd.Timedelta(hours=12)
transporting.loc[~transporting_filter, 'N+'] = transporting.loc[~transporting_filter]['N0'] + pd.Timedelta(hours=32)
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn'])) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) | ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))) |\
        ((data['TrangThai'] == 'Đang trung chuyển hàng hoàn') & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) | ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting_filter = (r_transporting['DenTinh'] == 'Hà Nội') | (r_transporting['DenTinh'] == 'Hồ Chí Minh')
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting.loc[r_transporting_filter, 'N+'] = r_transporting.loc[r_transporting_filter]['N0'] + pd.Timedelta(hours=36)
r_transporting.loc[~r_transporting_filter, 'N+'] = r_transporting.loc[~r_transporting_filter]['N0'] + pd.Timedelta(hours=56)
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# convert Aging to hours
data['Aging (hours)'] = data['Aging']/np.timedelta64(1, 'h')
# set e-commerces column to NaN
data['e-commerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'


# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
# reset the index of dataframe
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,Aging (hours),e-commerces
0,8FH1DDY73A,S1046057O3263,2020-08-13 14:39:19,2020-08-13 14:39:19,2020-08-21 19:35:42,NaT,NaT,2020-08-25 15:54:00,NaT,NaN,Quận Bắc Từ Liêm,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Kho Khách hàng lớn Hà Nội,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hà Nội,Lưu kho,10,10x5x5,2532447,'13/08/2020 14:36:37 - GHN-PFA1A0|THAY ĐỔI THÔ...,'V1506SIZE: M',2 Ngày,8,0,0,NaN,2031833,NaN,'CAM ANH BOUTIQUE',1270558,20FDA80687,2329.0,2275.0,2020-08-23 20:23:45,Kết thúc,Kho giao,2020-08-23 20:23:45,2020-08-28 20:23:45,-3 days +16:36:35.629173,-55.390103,NaN
1,8FH3DD3FH9,S1600925O546,2020-08-13 16:26:07,2020-08-14 08:00:00,2020-08-15 14:14:44,2020-08-17 08:34:26,NaT,2020-08-19 01:06:00,NaT,NaN,Quận Cầu Giấy,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Đang giao hàng,10,10x5x5,1301102,'14/08/2020 08:06:07 - GHN-PFA1A0|THAY ĐỔI THÔ...,'',2 Ngày,3,3,0,NaN,1874751,NaN,'Liinh Kids',1274017,20ECT63012,2329.0,2275.0,2020-08-16 14:15:18,Kết thúc,Kho giao,2020-08-16 14:15:18,2020-08-21 14:15:18,4 days 22:45:02.629173,118.750730,NaN
2,8FHSAHAY5K,KV1293213386351,2020-08-13 20:36:59,2020-08-14 08:00:00,2020-08-14 19:07:15,2020-08-17 07:55:13,NaT,2020-08-19 01:06:00,NaT,NaN,Quận Hoàn Kiếm,Hà Nội,G,Quận 4,Hồ Chí Minh,E,Bưu Cục 55C Nguyễn Khoái-Q.Hai Bà Trưng-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 55C Nguyễn Khoái-Q.Hai Bà Trưng-HN,Giao hàng không thành công,500,1x1x1,518873,'17/08/2020 06:39:33 - GHN-DFC1A1|Khách đổi đị...,'',2 Ngày,1,1,0,NaN,1883870,NaN,'maiphuongbuiboutique',1228196,20WMZ58349,2329.0,2275.0,2020-08-16 14:17:02,Kết thúc,Kho giao,2020-08-16 14:17:02,2020-08-21 14:17:02,4 days 22:43:18.629173,118.721841,NaN
3,FHPAFQS4,KV1293243226366,2020-08-01 10:57:01,2020-08-01 10:57:00,2020-08-01 16:10:36,2020-08-04 09:17:38,NaT,2020-08-04 23:00:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 02,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Đang giao hàng,500,1x1x1,518873,NaN,NaN,2 Ngày,1,1,0,0,1714228,9999,'ptlink',2267105,20NRU94012,2329.0,2275.0,2020-08-03 14:07:25,Kết thúc,Kho giao,2020-08-03 14:07:25,2020-08-08 14:07:25,17 days 22:52:55.629173,430.882119,NaN
4,8FHNL3F46,NaN,2020-08-04 15:42:45,2020-08-04 15:42:45,2020-08-04 16:26:44,2020-08-05 08:23:43,NaT,2020-08-08 23:00:00,NaT,NaN,Huyện Hớn Quản,Bình Phước,D,Quận 7,Hồ Chí Minh,E,Bưu Cục Chơn Thành-Bình Phước,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Chơn Thành-Bình Phước,Lưu kho,500,10x10x10,1045101,'05/08/2020 03:39:43 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',2 Ngày,1,1,0,NaN,1883501,NaN,'St White',949270,20DMK99349,2329.0,2275.0,2020-08-05 03:12:50,Kết thúc,Kho giao,2020-08-05 03:12:50,2020-08-10 03:12:50,16 days 09:47:30.629173,393.791841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6006,8FHFN3ARF4,5f3a288d3e048f0001baa624_136,2020-08-17 14:27:38,2020-08-17 14:27:38,2020-08-18 11:42:23,2020-08-21 10:57:16,2020-08-24 13:29:34,2020-08-22 23:09:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Huyện Đan Phượng,Hà Nội,G,Quận 4,Hồ Chí Minh,E,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Chuyển hoàn,2000,10x10

In [53]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging (hours)',
    'e-commerces',
    'TrangThaiLuanChuyen',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_everyday'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_everyday' id:181055436>

In [54]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val

In [55]:
# to excel
# data_to_excel = data[[
#     'MaDH',
#     'KhoHienTai',
#     'TrangThai',
#     'GhiChuGHN',
#     'SoLanLay',
#     'SoLanGiao',
#     'SoLanTra',
#     'LoaiBacklog',
#     'N0',
#     'N+',
#     'Aging',
#     'Aging (hours)',
#     'e-commerces',
# ]]

# data_to_excel.to_excel('backlog.excel', sheet='Sheet1')